<a href="https://colab.research.google.com/github/codyhsu/investment-pipeline/blob/main/101_Investment_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance pandas google-cloud-bigquery pandas-gbq

#**Load Transaction Data**

In [ ]:
import pandas as pd, gdown

# Load transaction data from google drive sharing link
file_id = "100DlxpNsKOuS2MH6gWptI0KkGHpJyZnE"
url = f"https://drive.google.com/uc?id={file_id}"
output = "investment_transactions_36_months.csv"
gdown.download(url, output, quiet=False)

# Read the CSV file with semicolon as a delimiter
transactions = pd.read_csv(output, sep=';')

# Convert the 'date' column to datetime objects, specifying the format
transactions['date'] = pd.to_datetime(transactions['date'], format='%d/%m/%Y')

display(transactions.head())

Downloading...
From: https://drive.google.com/uc?id=100DlxpNsKOuS2MH6gWptI0KkGHpJyZnE
To: /content/investment_transactions_36_months.csv
100%|██████████| 3.42k/3.42k [00:00<00:00, 8.24MB/s]


,date,ticker,amount_eu
0,2022-11-01,SPY,120
1,2022-11-01,BNDX,60
2,2022-11-01,VGK,60
3,2022-11-01,IEF,45
4,2022-11-01,SHV,15


# **Load Yahoo Finance Data**


In [ ]:
import yfinance as yf
import pandas as pd

# Identify tickers, start and end date form transaction
tickers = transactions.ticker.unique().tolist()
start = transactions.date.min()
end = transactions.date.max() + pd.Timedelta(days=7) # Extend end date by 7 days
# include market index
market_indices = ['^GSPC']
all_tickers = market_indices + tickers
#import desired tickers from yahoo finance
raw_yf_data = yf.download(all_tickers, start=start, end=end)
raw_yf_data.head()

/tmp/ipython-input-3471694711.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw_yf_data = yf.download(all_tickers, start=start, end=end)
[*********************100%***********************]  6 of 6 completed


Price           Close                                               \
Ticker           BNDX        IEF        SHV         SPY        VGK   
Date                                                                 
2022-11-01  42.808643  85.186890  95.672401  369.023376  45.820084   
2022-11-02  42.755028  85.042198  95.672401  359.762299  44.993103   
2022-11-03  42.620983  84.671379  95.715996  356.057922  44.520546   
2022-11-04  42.594181  84.490479  95.724716  361.182678  46.565277   
2022-11-07  42.451225  84.192024  95.733406  364.637604  46.810646   

Price                         High                                    ...  \
Ticker            ^GSPC       BNDX        IEF        SHV         SPY  ...   
Date                                                                  ...   
2022-11-01  3856.100098  42.915865  85.910418  95.689832  374.656833  ...   
2022-11-02  3759.689941  42.960541  85.801911  95.689832  372.967764  ...   
2022-11-03  3719.889893  42.674592  84.825128  95.715996  359.119364  ...   
2022-11-04  3770.550049  42.692469  84.969818  95.733435  363.601109  ...   
2022-11-07  3806.800049  42.620997  84.644232  95.733406  365.232612  ...   

Price            Open                                       Volume           \
Ticker            SHV         SPY        VGK        ^GSPC     BNDX      IEF   
Date                                                                          
2022-11-01  95.672401  374.416908  46.392608  3901.790039  2986100  6066400   
2022-11-02  95.681113  368.428378  45.756468  3852.899902  2996600  7581400   
2022-11-03  95.707284  356.499376  44.257002  3733.250000  2706900  8403900   
2022-11-04  95.715998  361.806477  46.092717  3766.979980  2621400  5287000   
2022-11-07  95.733406  362.487858  46.792474  3780.709961  2247600  4289300   

Price                                                 
Ticker           SHV        SPY      VGK       ^GSPC  
Date                                                  
2022-11-01  21541000   85407600  5162500  4481210000  
2022-11-02   1911100  126990400  5927100  4899000000  
2022-11-03   2998000   87100100  4129900  4625290000  
2022-11-04   2363200  103505200  6688000  5400180000  
2022-11-07   3297300   68286900  3818200  4341620000  

[5 rows x 30 columns]

In [ ]:
yf_data = raw_yf_data.xs('Close', level=0, axis=1).stack().reset_index()
yf_data.columns = ['date', 'ticker', 'price_usd']
yf_data['date'] = pd.to_datetime(yf_data['date']) # Corrected: Convert the 'date' column within yf_data itself
display(yf_data.head())

,date,ticker,price_usd
0,2022-11-01,BNDX,42.808643
1,2022-11-01,IEF,85.186890
2,2022-11-01,SHV,95.672401
3,2022-11-01,SPY,369.023376
4,2022-11-01,VGK,45.820084


In [ ]:
benchmark = yf_data[yf_data['ticker'] == '^GSPC'].rename(columns={'price_usd': 'sp500_usd'})
etf_prices = yf_data[yf_data['ticker'] != '^GSPC']
display(benchmark.head())
display(etf_prices.head())

,date,ticker,sp500_usd
5,2022-11-01,^GSPC,3856.100098
11,2022-11-02,^GSPC,3759.689941
17,2022-11-03,^GSPC,3719.889893
23,2022-11-04,^GSPC,3770.550049
29,2022-11-07,^GSPC,3806.800049


,date,ticker,price_usd
0,2022-11-01,BNDX,42.808643
1,2022-11-01,IEF,85.186890
2,2022-11-01,SHV,95.672401
3,2022-11-01,SPY,369.023376
4,2022-11-01,VGK,45.820084


Load Excahnge Rate EUR vs USD

In [ ]:
import pandas as pd
import yfinance as yf

eur_usd_ticker = 'EURUSD=X'
start_date = transactions['date'].min()
end_date = transactions['date'].max() + pd.Timedelta(days=7) # Extend end date by 7 days

eur_usd_data = yf.download(eur_usd_ticker, start=start_date, end=end_date)
eur_usd_data = eur_usd_data[['Close']].reset_index()
eur_usd_data.columns = ['date', 'EUR_USD_rate']

display(eur_usd_data.head())

/tmp/ipython-input-3786520456.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  eur_usd_data = yf.download(eur_usd_ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


,date,EUR_USD_rate
0,2022-11-01,0.988631
1,2022-11-02,0.987791
2,2022-11-03,0.981441
3,2022-11-04,0.974963
4,2022-11-07,0.992881


In [ ]:
etf_prices.to_csv("etf_prices.csv", index=False)
benchmark.to_csv("benchmark.csv", index=False)
eur_usd_data.to_csv("fx_rate.csv", index=False)

In [ ]:
''' download to desktop to confirm
from google.colab import files
files.download("etf_prices.csv")
files.download("benchmark.csv")
files.download("fx_rate.csv")
'''

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

FileNotFoundError: Cannot find file: transactions.csv

In [ ]:
from google.colab import auth
from pandas_gbq import to_gbq

auth.authenticate_user()
project_id = "investment-pipeline-477615"

to_gbq(etf_prices, "portfolio.market_prices", project_id=project_id, if_exists="replace")
to_gbq(transactions, "portfolio.transactions_raw", project_id=project_id, if_exists="replace")
to_gbq(eur_usd_data, "portfolio.fx_rates", project_id=project_id, if_exists="replace")
to_gbq(benchmark[['date', 'sp500_usd']], "portfolio.benchmark_sp500", project_id=project_id, if_exists="replace")

print("Data uploaded to BigQuery!")

100%|██████████| 1/1 [00:00<00:00, 12446.01it/s]

Data uploaded to BigQuery!
